In [38]:
import pandas as pd
import numpy as np
import random

def clean_separate_df(df, yr):
    all_players = df[df.year==yr][['player','pos','adp','ppr_pts','year']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te


def create_strats(strat):
    if strat == 'h':
        return ['hero_rb', ['RB','WR','TE','QB','WR','WR','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'z':
        return ['zero_rb', ['WR','WR','TE','QB','VAL','VAL','RB','RB','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'r':
        return ['rb_heavy', ['RB','RB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    else:
        return ['val',['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    
def create_owners_strats(hero,zero,rb_heavy,value):
    strats = []
    for i in range(0,hero):
        strats.append(create_strats('hero'))
    for i in range(0,zero):
        strats.append(create_strats('zero'))
    for i in range(0,rb_heavy):
        strats.append(create_strats('rb_heavy'))
    for i in range(0,value):
        strats.append(create_strats('value'))
    return strats

def one_random_strat():
    '''h - hero, z - zero, r - rb_heavy, v - value'''
    return random.choice(['h','z','r','v','v','v'])

def create_league():
    league_draft_strat= []
    for i in range(0,12):
        league_draft_strat.append(one_random_strat())
        
    strats = []
    for l in league_draft_strat:
        strats.append(create_strats(l))
        
    return strats

class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos, draft_num):
        '''
        strat - draft strategy as tuple
        draft_pos - draft position 1-12
        draft_num - represents a unique league per season
        '''
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.draft_num = draft_num
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 13 and self.te_count == 0: #Draft a TE in 10th round if no TEs
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif len(self.team) == 8 and self.qb_count == 0: #Draft a QB in 9th round if no QBs
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        elif len(self.team) == 7 and self.rb_count == 1: #Draft a RB in 8th round if only 1 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 1: #Draft a WR in 8th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif len(self.team) == 6 and self.rb_count == 0: #Draft a RB in 7th round if only 0 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 0: #Draft a WR in 7th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL': #Don't draft more than 2 QB
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL': # Don't draft more than 2 TE
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        
#         elif self.rb_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_wr(self)
        
#         elif self.wr_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_rb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)



            
    

In [2]:
def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top = []
    for season in range(2010,2023):
        for sim in range(0,100):
            for i in range(1,13):
                top7 = [] 
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='QB')].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & ((df.pos=='RB')|(df.pos=='WR'))].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='TE')].sort_values('ppr_pts',ascending=False).head(1))
                top.append(pd.concat(top7))
    return pd.concat(top)

# Create 100 mock leagues each year to run analysis on

In [3]:
df = pd.read_csv('season1.csv',index_col=0)
owners_teams = [] 
for season in range(2010,2023):
    for sim in range(0,100):
        all_players, qb, rb, wr, te = clean_separate_df(df,season)
        strats = create_league()
        cheat = Cheatsheet(all_players, qb, rb, wr, te)

        owners = [] #Create 12 team owners with 'value_strat' drafting strategy
        for i, s in zip(range(0,12),strats.copy()):
            owners.append(Team_owner(s,i+1,sim))

        owners.reverse()
        for i in range(0,14): #Draft 14 rounds for each team owner
            owners.reverse()
            for o in owners:
                o.team.append(o.draft_player(cheat))

        owners_teams.append(owners)

all_teams = []
for t in owners_teams:
    for o in t:
        df = pd.concat(o.team,axis=1).T
        df['draft_pos'] = o.draft_pos
        df['draft_strat'] = o.strat_name
        df['league_id'] = o.draft_num
        all_teams.append(df) 

df = pd.concat(all_teams)
    
   

In [4]:
def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    df = df.reset_index(drop=True)
    df.index+=1
    df['pick_number'] = df.index
    
    top7 = [] 
    top7.append(df[df.pos=='QB'].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[(df.pos=='RB')|(df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='TE'].sort_values('ppr_pts',ascending=False).head(1))
    df = pd.concat(top7)
    df['total_pts'] = df.ppr_pts.sum()
    return df

In [5]:
com = []
for seas in range(2010,2023): # for 'year'
    for num in range(0,100): #for 'draft_num'
        for i in range(1,13): #for 'draft_pos'
            temp = df[(df.year==seas)&(df.draft_num==num)&(df.draft_pos==i)]
            com.append(top7(temp))
top = pd.concat(com) #df of each team owners top 7 starters with 'total_pts' column

In [6]:
top.to_csv('top7.csv')
df.to_csv('sim100.csv')

In [39]:
df.head(1)

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
11,Calvin Johnson,WR,12.0,266.2,2010,12,val,0


In [43]:
top[(top.pos=='TE')&(top.player_pick==1)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num,player_pick,total_pts
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,0,1,1371.4
1,Jimmy Graham,TE,8.0,229.9,2014,11,val,1,1,1444.9
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,2,1,1410.0
1,Jimmy Graham,TE,8.0,229.9,2014,9,val,3,1,1500.2
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,4,1,1371.4
...,...,...,...,...,...,...,...,...,...,...
1,Travis Kelce,TE,8.0,262.8,2021,10,val,95,1,1588.9
1,Travis Kelce,TE,8.0,262.8,2021,10,val,96,1,1847.2
1,Travis Kelce,TE,8.0,262.8,2021,10,val,97,1,1764.7
1,Travis Kelce,TE,8.0,262.8,2021,9,val,98,1,1702.7


In [47]:
df[(df.year==2014)&(df.draft_num==0)&(df.draft_pos==8)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
7,Jimmy Graham,TE,8.0,229.9,2014,8,val,0
16,Giovani Bernard,RB,17.0,187.9,2014,8,val,0
31,Roddy White,WR,33.0,212.1,2014,8,val,0
38,Pierre Garcon,WR,40.0,161.2,2014,8,val,0
55,Torrey Smith,WR,58.0,191.7,2014,8,val,0
64,Bishop Sankey,RB,67.0,96.2,2014,8,val,0
79,Jay Cutler,QB,82.0,255.6,2014,8,val,0
87,Greg Olsen,TE,90.0,220.8,2014,8,val,0
103,Sammy Watkins,WR,107.0,198.0,2014,8,val,0
112,DeAngelo Williams,RB,118.0,29.3,2014,8,val,0


In [54]:
top[(top.pos=='TE')&(top.player_pick==1)&(top.draft_pos<9)]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num,player_pick,total_pts
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,0,1,1371.4
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,2,1,1410.0
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,4,1,1371.4
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,6,1,1252.9
1,Jimmy Graham,TE,8.0,229.9,2014,8,val,7,1,1315.2
...,...,...,...,...,...,...,...,...,...,...
1,Travis Kelce,TE,8.0,262.8,2021,8,val,71,1,1526.9
1,Travis Kelce,TE,8.0,262.8,2021,8,val,76,1,1561.3
1,Travis Kelce,TE,8.0,262.8,2021,8,val,85,1,1600.8
1,Travis Kelce,TE,8.0,262.8,2021,8,val,87,1,1405.7


In [8]:
top.total_pts.mean()

1495.3744262084417

In [9]:
top.groupby('draft_strat')['total_pts'].mean().sort_values()

draft_strat
rb_heavy    1465.698781
val         1496.535617
zero_rb     1505.888285
hero_rb     1510.876911
Name: total_pts, dtype: float64

In [10]:
top.groupby('draft_pos')['total_pts'].mean()

draft_pos
1     1509.803858
2     1524.259538
3     1511.927563
4     1514.800011
5     1514.163304
6     1504.143130
7     1484.446302
8     1483.843581
9     1496.196724
10    1493.808846
11    1452.889769
12    1454.218231
Name: total_pts, dtype: float64

In [82]:
top[(top.draft_pos==5)&(top.draft_strat=='zero_rb')&(top.year==2020)].head(35)

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num,player_pick,total_pts
4,Russell Wilson,QB,65.0,359.8,2020,5,zero_rb,8,4,1386.6
7,Antonio Gibson,RB,80.0,202.2,2020,5,zero_rb,8,7,1386.6
8,Latavius Murray,RB,97.0,136.2,2020,5,zero_rb,8,8,1386.6
2,Chris Godwin,WR,22.0,191.0,2020,5,zero_rb,8,2,1386.6
9,Emmanuel Sanders,WR,103.0,164.8,2020,5,zero_rb,8,9,1386.6
10,Sterling Shepard,WR,122.0,162.5,2020,5,zero_rb,8,10,1386.6
3,Mark Andrews,TE,43.0,170.1,2020,5,zero_rb,8,3,1386.6
4,Kyler Murray,QB,66.0,378.7,2020,5,zero_rb,10,4,1244.6
7,Zack Moss,RB,85.0,101.6,2020,5,zero_rb,10,7,1244.6
11,Duke Johnson,RB,137.0,84.4,2020,5,zero_rb,10,11,1244.6


In [81]:
df[(df.draft_pos==5)&(df.draft_strat=='zero_rb')&(df.year==2020)].head(28)

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
4,Michael Thomas,WR,5.0,83.9,2020,5,zero_rb,8
21,Chris Godwin,WR,22.0,191.0,2020,5,zero_rb,8
40,Mark Andrews,TE,43.0,170.1,2020,5,zero_rb,8
61,Russell Wilson,QB,65.0,359.8,2020,5,zero_rb,8
50,Cam Akers,RB,54.0,101.8,2020,5,zero_rb,8
67,A.J. Green,WR,71.0,111.3,2020,5,zero_rb,8
76,Antonio Gibson,RB,80.0,202.2,2020,5,zero_rb,8
93,Latavius Murray,RB,97.0,136.2,2020,5,zero_rb,8
99,Emmanuel Sanders,WR,103.0,164.8,2020,5,zero_rb,8
115,Sterling Shepard,WR,122.0,162.5,2020,5,zero_rb,8


In [11]:
top.groupby(['draft_pos','draft_strat'])[['total_pts']].mean().sort_values(['draft_pos','total_pts'])

total_pts
draft_pos draft_strat             
1         rb_heavy     1472.043689
          val          1506.076247
          zero_rb      1526.161404
          hero_rb      1538.558929
2         rb_heavy     1491.755696
          zero_rb      1523.342727
          val          1533.367656
          hero_rb      1534.485714
3         rb_heavy     1478.245894
          val          1514.866279
          zero_rb      1520.240670
          hero_rb      1525.630612
4         rb_heavy     1469.761215
          zero_rb      1516.803256
          val          1519.965281
          hero_rb      1545.149198
5         hero_rb      1499.846429
          rb_heavy     1503.229952
          val          1516.001905
          zero_rb      1534.531579
6         rb_heavy     1481.386312
          hero_rb      1499.897131
          zero_rb      1500.952743
          val          1515.040819
7         rb_heavy     1462.127033
          val          1467.979268
          zero_rb      1519.846694
          hero_rb      1520.790052
8         rb_heavy     1463.228946
          val          1479.911674
          zero_rb      1487.944889
          hero_rb      1512.383412
9         rb_heavy     1470.787354
          val          1500.560951
          zero_rb      1502.868692
          hero_rb      1504.251208
10        rb_heavy     1463.784120
          zero_rb      1480.418947
          hero_rb      1500.998157
          val          1505.899394
11        rb_heavy     1420.796209
          val          1449.835909
          zero_rb      1471.640580
          hero_rb      1474.988288
12        rb_heavy     1407.598551
          val          1450.456394
          zero_rb      1480.928241
          hero_rb      1481.947807

In [12]:
top.groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

mean  count
draft_pos draft_strat                    
4         hero_rb      1545.149198   1309
1         hero_rb      1538.558929   1568
5         zero_rb      1534.531579   1463
2         hero_rb      1534.485714   1421
          val          1533.367656   4480
1         zero_rb      1526.161404   1596
3         hero_rb      1525.630612   1715
2         zero_rb      1523.342727   1540
7         hero_rb      1520.790052   1337
3         zero_rb      1520.240670   1463
4         val          1519.965281   4787
7         zero_rb      1519.846694   1694
4         zero_rb      1516.803256   1505
5         val          1516.001905   4619
6         val          1515.040819   4248
3         val          1514.866279   4472
8         hero_rb      1512.383412   1477
1         val          1506.076247   4492
10        val          1505.899394   4620
9         hero_rb      1504.251208   1449
5         rb_heavy     1503.229952   1449
9         zero_rb      1502.868692   1498
10        hero_rb      1500.998157   1519
6         zero_rb      1500.952743   1659
9         val          1500.560951   4520
6         hero_rb      1499.897131   1708
5         hero_rb      1499.846429   1568
2         rb_heavy     1491.755696   1659
8         zero_rb      1487.944889   1575
12        hero_rb      1481.947807   1596
6         rb_heavy     1481.386312   1483
12        zero_rb      1480.928241   1512
10        zero_rb      1480.418947   1330
8         val          1479.911674   4557
3         rb_heavy     1478.245894   1449
11        hero_rb      1474.988288   1554
1         rb_heavy     1472.043689   1442
11        zero_rb      1471.640580   1449
9         rb_heavy     1470.787354   1629
4         rb_heavy     1469.761215   1498
7         val          1467.979268   4592
10        rb_heavy     1463.784120   1631
8         rb_heavy     1463.228946   1489
7         rb_heavy     1462.127033   1476
12        val          1450.456394   4543
11        val          1449.835909   4620
          rb_heavy     1420.796209   1477
12        rb_heavy     1407.598551   1449

In [13]:
top[top.draft_strat=='hero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count','max','min']).\
sort_values('mean',ascending=False)

,,mean,count,max,min
draft_pos,draft_strat,,,,
4,hero_rb,1545.149198,1309,1947.7,1174.7
1,hero_rb,1538.558929,1568,1984.1,1071.4
2,hero_rb,1534.485714,1421,1904.7,1101.8
3,hero_rb,1525.630612,1715,2035.7,1070.6
7,hero_rb,1520.790052,1337,1985.4,1092.8
8,hero_rb,1512.383412,1477,2028.7,1036.6
9,hero_rb,1504.251208,1449,2027.8,1045.1
10,hero_rb,1500.998157,1519,2037.5,977.2
6,hero_rb,1499.897131,1708,1972.1,1044.5


In [14]:
top[top.draft_strat=='zero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
5,zero_rb,1534.531579,1463
1,zero_rb,1526.161404,1596
2,zero_rb,1523.342727,1540
3,zero_rb,1520.240670,1463
7,zero_rb,1519.846694,1694
4,zero_rb,1516.803256,1505
9,zero_rb,1502.868692,1498
6,zero_rb,1500.952743,1659
8,zero_rb,1487.944889,1575


In [15]:
top[top.draft_strat=='val'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
2,val,1533.367656,4480
4,val,1519.965281,4787
5,val,1516.001905,4619
6,val,1515.040819,4248
3,val,1514.866279,4472
1,val,1506.076247,4492
10,val,1505.899394,4620
9,val,1500.560951,4520
8,val,1479.911674,4557


In [16]:
top[top.draft_strat=='rb_heavy'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
5,rb_heavy,1503.229952,1449
2,rb_heavy,1491.755696,1659
6,rb_heavy,1481.386312,1483
3,rb_heavy,1478.245894,1449
1,rb_heavy,1472.043689,1442
9,rb_heavy,1470.787354,1629
4,rb_heavy,1469.761215,1498
10,rb_heavy,1463.784120,1631
8,rb_heavy,1463.228946,1489


In [20]:
top.groupby(['player_pick','pos'])['total_pts'].agg(['mean','count'])

mean  count
player_pick pos                    
1           QB   1387.763059    425
            RB   1512.698358   8342
            TE   1538.721942    278
            WR   1509.097283   4196
2           QB   1492.692138    814
            RB   1489.584320   4694
            TE   1521.098698    691
            WR   1510.928841   6314
3           QB   1505.417207    895
            RB   1481.656538   2975
            TE   1518.753147   4051
            WR   1492.306669   3284
4           QB   1509.573741   5381
            RB   1473.589342   2749
            TE   1520.354930     71
            WR   1501.549396   4221
5           QB   1468.694972    179
            RB   1484.055000   3100
            TE   1526.917785    298
            WR   1508.498641   5594
6           QB   1497.836472    924
            RB   1472.940360   2500
            TE   1524.275769    780
            WR   1488.237013   4077
7           QB   1477.251791    977
            RB   1486.620031   3180
            TE   1475.143517    779
            WR   1494.807111   2489
8           QB   1504.606136   1092
            RB   1507.602802   3498
            TE   1505.631501   1146
            WR   1467.846193   2758
9           QB   1471.404651   2838
            RB   1505.999728   1101
            TE   1476.912759    627
            WR   1506.194860   2101
10          QB   1513.438602    601
            RB   1475.235335   1282
            TE   1500.446035   1034
            WR   1481.223691   1967
11          QB   1517.878144    668
            RB   1534.461714    922
            TE   1486.206623    604
            WR   1499.831669   1803
12          QB   1517.166590    437
            RB   1460.483175    844
            TE   1515.003593   1475
            WR   1478.238544   1030
13          QB   1507.875748    301
            RB   1482.966140   1013
            TE   1554.203941   1015
            WR   1459.845586   1110
14          QB   1510.630882     68
            RB   1537.601399    286
            TE   1411.221992   2751
            WR   1466.479137    556

In [64]:
top.head(1)

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num,player_pick,total_pts
4,Brett Favre,QB,87.0,97.2,2010,1,hero_rb,0,4,1225.6


In [69]:
len(top[(top.draft_strat=='hero_rb')&(top.draft_pos==3)])

1715

In [68]:
top[(top.draft_strat=='hero_rb')&(top.draft_pos==3)].groupby(['draft_pos','player_pick','pos'])['total_pts'].agg(['mean','count'])

mean  count
draft_pos player_pick pos                    
3         1           RB   1530.339827    231
          2           WR   1548.301990    201
          3           TE   1538.919209    177
          4           QB   1525.630612    245
          5           WR   1523.295676    185
          6           WR   1479.044056    143
          7           RB   1507.732258     62
                      TE   1453.400000     11
                      WR   1520.734375     32
          8           RB   1541.452500     80
                      TE   1455.100000      7
                      WR   1665.747619     21
          9           RB   1613.268889     45
                      TE   1371.420000      5
                      WR   1535.770588     34
          10          RB   1409.828571      7
                      TE   1509.977778      9
                      WR   1497.618367     49
          11          RB   1531.327778     18
                      TE   1430.900000      5
                      WR   1545.183333     12
          12          RB   1466.900000     25
                      TE   1656.715385     13
                      WR   1550.743750     16
          13          RB   1460.657500     40
                      TE   1448.833333     18
                      WR   1524.614286     14
          14          RB   1604.383333      6
                      WR   1343.600000      4